# Monte Carlo Calibration via Deep Learning

## Sample code to illustrate AMCC in a Black-Scholes model

In [ ]:
import torch
import numpy as np
from torch.distributions import Normal

A first step for AMCC consists in implementing the forward simulation scheme of the underlying market model. In the Black-Scholes model we could directly use the closed-form expression (see equation (1) in the paper). Here we compute the log-prices iteratively, in order to create the link to more complex models.

In [ ]:
def simulated_paths_new(states_in,path_bm,vol,maturity,drift):
    '''
    Generate sample paths of a Black-Scholes model for given initial states (states_in),
    Brownian paths (path_bm), volatility, maturity and drift.
    '''
    vec_dim = path_bm.shape
    time_steps = vec_dim[1]
    h = maturity/time_steps
    log_states = torch.log(states_in)

    for i in range(time_steps):

        brownian_incr = path_bm[:,i]
        log_states  = log_states + (drift-0.5*vol**2)*h+vol*brownian_incr

    return torch.exp(log_states)

Next we implement the Monte Carlo pricing function (see equation (1) in the paper). We implement antithetic sampling to reduce the number of required samples.

In [ ]:
def payoff(strikes,samples):
    '''
    Evaluate call option payoff on given input samples and for given strikes
    '''
    diff_tmp = samples.reshape(-1,1)-strikes
    diff_tmp [diff_tmp  < 0] = 0

    return diff_tmp

def BS_monte_carlo_price(strikes,samples,samples_anti):
    '''
    Compute Monte Carlo price for given strikes, samples and associated antithetic samples
    '''
    price = torch.mean(torch.transpose(payoff(strikes,samples),0,1),dim=1)
    price_anti = torch.mean(torch.transpose(payoff(strikes,samples_anti),0,1),dim=1)

    return (price+price_anti)/2

Next, we implement the Monte Carlo pricing loss in torch and optimize over the volatility parameter. The working principle is the same as when training neural networks. However, no neural networks explicitly appear here.

In [ ]:
def AMCC_optim(strikes,call_prices,vol,iter_numb,learning_rate,maturity,batch_size,time_steps,drift,S0):
    '''
    Implements AMCC for "market" call prices with given strikes to calibrate
    the vol parameter in a Black Scholes model for given drift and initial value S0.
    Parameter optimization is performed using Adam optimizer for iter_numb iterations, with specified
    learning_rate and batch_size.
    '''
    vol.requires_grad = True
    optimizer = torch.optim.Adam([vol], lr=learning_rate)
    h = maturity/time_steps

    for itr in range(iter_numb):

        init_state  = torch.ones(batch_size)*S0
        paths_bm = torch.normal(0, 1, (batch_size,time_steps))*torch.sqrt(h)

        samples = simulated_paths_new(init_state,paths_bm,vol,maturity,drift)
        samples_anti = simulated_paths_new(init_state,-paths_bm,vol,maturity,drift)

        model_prices = BS_monte_carlo_price(strikes,samples,samples_anti)
        loss =torch.mean((model_prices - call_prices)**2,dim=1)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if itr%100==0:
            print("Epoch number: {} and the loss : {}".format(itr,loss.item()))

    return vol

In [ ]:
std_norm_cdf = Normal(0, 1).cdf
std_norm_pdf = lambda x: torch.exp(Normal(0, 1).log_prob(x))

def bs_price(K, S, T, sigma, r):
    d_1 = (1 / (sigma * torch.sqrt(T))) * (torch.log(S / K) + (r + (torch.square(sigma) / 2)) * T)
    d_2 = d_1 - sigma * torch.sqrt(T)

    C = std_norm_cdf(d_1) * S - std_norm_cdf(d_2) * K * torch.exp(-r * T)
    return C


We illustrate the use of AMCC in a simple example. We generate artificial strikes and "market prices" from a Black-Scholes model with given *true* volatility 0.05.

Then we use AMCC to calibrate a Black-Scholes model (with a different initial volatility 0.01) to these option prices.

In [ ]:
# Black Scholes model specifications
S0 = 100.
drift = torch.tensor(0.00)
# True vol:
vol_true = torch.tensor(0.05)
# Starting point for vol calibration
vol = torch.tensor(0.01)

# Generate artificial price data: 20 options with different strikes
maturity = torch.tensor(1.)
time_steps = 5
numb_strikes = 20
strikes_test = torch.zeros(1,numb_strikes)
# Select strikes
for i in range(numb_strikes):
    strikes_test[0,i] = 95 + i*1
# Compute Black-Scholes price for each strike
call_prices = torch.zeros(1,numb_strikes)
for i in range(numb_strikes):
    call_prices[0,i] = bs_price(strikes_test[0,i], S0, maturity, vol_true, drift)

# Now we do Monte Carlo calibration and try to recover vol_true
iter_numb = 2000
learning_rate = 0.0001
batch_size = 30000
vol_calibrated = AMCC_optim(strikes_test,call_prices,vol,iter_numb,learning_rate,maturity,batch_size,time_steps,drift,S0)

Epoch number: 0 and the loss : 0.6027946472167969
Epoch number: 100 and the loss : 0.408104807138443
Epoch number: 200 and the loss : 0.19724467396736145
Epoch number: 300 and the loss : 0.06861528009176254
Epoch number: 400 and the loss : 0.016503578051924706
Epoch number: 500 and the loss : 0.0025018430314958096
Epoch number: 600 and the loss : 4.382756378618069e-05
Epoch number: 700 and the loss : 5.193680408410728e-05
Epoch number: 800 and the loss : 7.376016583293676e-05
Epoch number: 900 and the loss : 6.839663456048584e-06
Epoch number: 1000 and the loss : 1.3728128578804899e-05
Epoch number: 1100 and the loss : 1.0675019439077005e-05
Epoch number: 1200 and the loss : 0.00015044597967062145
Epoch number: 1300 and the loss : 1.7297246813541278e-05
Epoch number: 1400 and the loss : 1.1005980923073366e-05
Epoch number: 1500 and the loss : 8.312280442623887e-06
Epoch number: 1600 and the loss : 1.4867769095872063e-05
Epoch number: 1700 and the loss : 4.59264174423879e-06
Epoch numbe

In [ ]:
torch.set_printoptions(precision=6)
print(vol_calibrated)

tensor(0.050004, requires_grad=True)


We started from initial volatility 0.01, but obtained a calibrated volatility parameter that is very close to the true value (0.05).